In [38]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
from pathlib import Path
from skimage import io
import numpy as np
import os

In [51]:
os.makedirs('data/tiles', exist_ok=True)
def create_tiles(path):
    image = io.imread(path)
    name = str(path).split('/')[-1].split('.')[0]
    pad_w, pad_h = 128 - (image.shape[0] % 128), 128 - (image.shape[1] % 128)
    image = np.pad(image, ((0, pad_w), (0, pad_h), (0, 0)))
    for w in range(0, image.shape[0], 128):
        for h in range(0, image.shape[1], 128):
            crop = image[w:w+128, h:h+128, :]
            io.imsave(f'data/tiles/{name}_w{w}_h{h}.png', crop)
            

In [52]:
!ls data/raw -1 | wc -l

9


In [62]:
%%timeit -r 1 -n 1
for path in Path('data/raw').glob('*'):
    create_tiles(path)
    

5.94 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [61]:
!rm data/tiles/*.png

In [60]:
%%timeit -r 1 -n 1
from concurrent.futures import ProcessPoolExecutor
with ProcessPoolExecutor() as executor:
    executor.map(create_tiles, Path('data/raw').glob('*'))
    

1.53 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
